# Analyzing Computer Science versus Business Management Introductory Course Professors Reviews and Their Trends Over Time

William Ingold, Erik Kelemen, Ashish Manda

## Introduction

## Grabbing Introductory Course Professors From UMD.io

In [1]:
import requests
professors_url = "https://api.umd.io/v1/professors"

In [5]:
def get_professors_for_courses(course_ids):
    professors = {}
    
    for course_id in course_ids:
        params = {'course_id': course_id}

        response = requests.get(professors_url, params)
        
        if response.status_code == 200:

            for item in response.json():
                name = item['name']

                if name in professors:
                    professors[name].add(course_id)
                else:
                    professors[name] = {course_id}

            return professors

### Computer Science Professors

In [6]:
cmsc_course_ids = ["CMSC131", "CMSC132", "CMSC216", "CMSC250"]
cmsc_professors = get_professors_for_courses(cmsc_course_ids)

if not cmsc_professors:
    print("Error response from umd.io API")

cmsc_professors.pop('Iason Filippou') # A typo of Jason Filippou from the database
print(cmsc_professors)

{'Evan Golub': {'CMSC131'}, 'Fawzi Emad': {'CMSC131', 'CMSC250', 'CMSC132'}, 'Ilchul Yoon': {'CMSC131', 'CMSC216', 'CMSC132'}, 'Nelson Padua-Perez': {'CMSC131', 'CMSC216', 'CMSC132'}, 'Pedram Sadeghian': {'CMSC131', 'CMSC132'}, 'Anwar Mamat': {'CMSC132'}, 'Laurence Herman': {'CMSC216', 'CMSC132'}, 'A Shankar': {'CMSC216'}, 'Aditya Acharya': {'CMSC250'}, 'Alexander Brassel': {'CMSC250'}, 'Clyde Kruskal': {'CMSC250'}, 'David Sekora': {'CMSC250'}, 'Donald Perlis': {'CMSC250'}, 'Jason Filippou': {'CMSC250'}, 'Mohammad Nayeem Teli': {'CMSC250'}, 'Roger Eastman': {'CMSC250'}}


### Business Management Professors

In [7]:
bmgt_course_ids = ["BMGT110", "BMGT220", "BMGT221", "BMGT230"]
bmgt_professors = get_professors_for_courses(bmgt_course_ids)

if not bmgt_professors:
    print("Error response from umd.io API")

print(bmgt_professors)

{'Hugh Turner': {'BMGT110'}, 'Jeff Miller': {'BMGT110'}, 'Cody Hyman': {'BMGT220'}, 'Laurel Mazur': {'BMGT220', 'BMGT221'}, 'Progyan Basu': {'BMGT220'}, 'Viktoriya Zotova': {'BMGT220'}, 'Gary Bulmash': {'BMGT221'}, 'Gerald Ward': {'BMGT221'}, 'Ai Ren': {'BMGT230'}, 'Daehoon Noh': {'BMGT230'}, 'Erich Studer-Ellis': {'BMGT230'}, 'Huan Cao': {'BMGT230'}, 'Radu Lazar': {'BMGT230'}, 'Shubham Akshat': {'BMGT230'}, 'Ziwei Cao': {'BMGT230'}}


## Grabbing Reviews

## From RateMyProfessor

In [33]:
from bs4 import BeautifulSoup

ratemyprofessor_url = "https://www.ratemyprofessors.com/search.jsp"

params = {'queryoption':'HEADER', 'schoolID':'1270', 'queryBy':'teacherName', 'schoolName':'University+of+Maryland'}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Headers": "Content-Type",
    "Access-Control-Allow-Methods": "GET"
}

In [119]:
def find_rmp_professor_url(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    #print(soup.find('li', class_='listing PROFESSOR').find("a", href=True)['href'])
    
    partial_url = soup.find('li', class_='listing PROFESSOR').find('a', href=True)
    
    if partial_url:
        main_url = "https://www.ratemyprofessors.com"
        return main_url + partial_url['href']

In [116]:
def query_rmp_for_professor(professor_name, headers, params):
    params['query'] = professor_name
    
    response = requests.get(ratemyprofessor_url, headers=headers, params=params)
    
    if response.status_code == 200:
        return find_rmp_professor_url(response.text)

#### Need Selenium because Javascript hides more reviews

In [138]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Firefox(executable_path='./bin/geckodriver.exe')

In [143]:
def load_all_rmp_reviews(page_url):
    driver.get(page_url)
    
    # RateMyProfessors has a Cookies pop up that overlays the website, it needs to be closed first
    time.sleep(0.5)
    close_cookies = driver.find_elements(By.XPATH, '//button[text()="Close"]')
    
    if close_cookies:
        close_cookies[0].click()
        
    load_more = driver.find_elements(By.XPATH, '//button[text()="Load More Ratings"]')
    
    # RateMyProfessors paginates the reviews via Javascript, so we must continually load more while the button is available
    while load_more:
        load_more[0].click()
        time.sleep(1)
        load_more = driver.find_elements(By.XPATH, '//button[text()="Load More Ratings"]')
        
    return driver.find_element_by_id('ratingsList').text

In [24]:
def parse_single_rmp_review(review_item):
    soup = BeautifulSoup(review_item, 'html.parser')
    course = soup.select('div[class*="RatingHeader__StyledClass"]').text
    print(course)
    

In [25]:
def get_all_rmp_reviews(review_html):
    soup = BeautifulSoup(review_html, 'html.parser')
    return None

In [26]:
def get_prof_rmp_stats(page_text):
    return None

### Computer Science Professors

In [144]:
fawzi_url = query_rmp_for_professor('Fawzi Emad', headers, params)
print(fawzi_url)
print(load_all_rmp_reviews(fawzi_url))

https://www.ratemyprofessors.com/ShowRatings.jsp?tid=313062
QUALITY
5.0
DIFFICULTY
1.0
CMSC131
😎
AWESOME
Dec 3rd, 2020
Would Take Again: Yes
Grade: A
Textbook: No
Online Class: Yes
Fawzi is a legend
HILARIOUS
LECTURE HEAVY
0
0
QUALITY
5.0
DIFFICULTY
2.0
CMSC131
😎
AWESOME
Dec 2nd, 2020
For Credit: Yes
Attendance: Mandatory
Would Take Again: Yes
Grade: Not sure yet
Textbook: No
Online Class: Yes
Fawzi is great. Lectures are super easy to follow, and he gives code examples for every lecture so you can practice yourself. Project specifications are really clear and the projects themselves are a piece of cake if you start at least a few days in advance. Quizzes are mostly super easy and exams are cake except for time constraints. Take Fawzi.
CLEAR GRADING CRITERIA
BEWARE OF POP QUIZZES
AMAZING LECTURES
0
0
QUALITY
4.0
DIFFICULTY
5.0
CMSC131
😎
AWESOME
Oct 23rd, 2020
For Credit: Yes
Attendance: Mandatory
Would Take Again: No
Grade: Not sure yet
Textbook: No
Good guy and teacher. Just gives a l

### Business Management Professors

## From Planetterp

### Computer Science Professors

### Business Management Professors